# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google - colab - secret 탭에 'OPENAI_API_KEY' 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 정확하고 전문적인 챗봇이야."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 나는 차은우야"
        }
      ]
    },
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ### => 아래처럼 예시 2개면 2-shot-prompting
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 테이의 FM 개꿀 라디오 방송에 주목해주세요.
- 교정 제목: 테이의 FM 라디오 방송에 주목하세요.


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
                }
            ],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말에 미친 개같은 폭우 예상.. 모두들 무사하시길 바래용'
output = correct_news_title(title_before, temperature=1, top_p=1)
print(output)

- 원래 제목: 주말에 미친 개같은 폭우 예상.. 모두들 무사하시길 바래용
- 교정 제목: 주말, 강한 폭우 예보… 모두 안전에 유의하시기 바랍니다.


##  영문텍스트에 anki 단어장 만들기

In [ ]:
# 함수 리팩토링

def anki_create(input_text, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 영어 단어 학습을 위한 Anki 단어장 생성 도우미입니다.

사용자가 제공한 영어 문장이나 단락에서 주요 어휘를 선별하고, 각 단어를 암기하기 좋은 카드 형태로 정리하세요.

### 카드 구성 방식 ###
각 단어에 대해 다음 정보를 포함하세요:

- 단어: [영단어]
- 의미: [한국어 의미]
- 품사: [명사/동사/형용사 등]
- 예문: [원문에서의 문장 또는 간단한 예시 문장]
- 발음기호(IPA): [가능하면]

### 출력 형식 ###
각 단어마다 아래와 같은 형식으로 출력하세요.

---
단어: benevolent
의미: 자비로운, 인정 많은
품사: 형용사
예문: She was a benevolent leader who helped everyone.
예문해석: 그녀는 모두를 도왔던 자비로운 리더였다.
발음기호: /bəˈnevələnt/
---

※ 너무 쉬운 단어(I, the, and 등)는 생략하세요.
※ 총 단어 수는 5~10개 이내로 제한하세요 (너무 많지 않게).
※ 예문에 대한 해석도 달아주세요.


    """

    user_message = f"""
    다음 영어 문장에서 주요 단어를 추출하고, Anki 단어장 형식으로 만들어 주세요.

텍스트:
{input_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
                },
                {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
                }
            ],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

input_text = """
"The government implemented new policies to mitigate the effects of climate change."

"Her benevolent nature made her beloved by everyone in the village."

"Despite the challenges, he remained resilient and optimistic."

"The manager emphasized the importance of clear communication in the workplace."

"This book offers a profound insight into the complexities of human behavior."

"""
output = anki_create(input_text, temperature=1, top_p=1)
print(output)

---
단어: implement  
의미: 시행하다, 실행하다  
품사: 동사  
예문: The government implemented new policies to mitigate the effects of climate change.  
예문해석: 정부는 기후 변화의 영향을 완화하기 위해 새로운 정책을 시행했다.  
발음기호: /ˈɪmplɪˌmɛnt/  
---

---
단어: mitigate  
의미: 완화하다, 경감하다  
품사: 동사  
예문: The government implemented new policies to mitigate the effects of climate change.  
예문해석: 정부는 기후 변화의 영향을 완화하기 위해 새로운 정책을 시행했다.  
발음기호: /ˈmɪtɪˌɡeɪt/  
---

---
단어: benevolent  
의미: 자비로운, 인정 많은  
품사: 형용사  
예문: Her benevolent nature made her beloved by everyone in the village.  
예문해석: 그녀의 자비로운 성격은 마을의 모든 사람이 그녀를 사랑하게 만들었다.  
발음기호: /bəˈnɛvələnt/  
---

---
단어: resilient  
의미: 회복력이 있는, 탄력 있는  
품사: 형용사  
예문: Despite the challenges, he remained resilient and optimistic.  
예문해석: 도전에도 불구하고 그는 회복력 있고 낙관적이었다.  
발음기호: /rɪˈzɪljənt/  
---

---
단어: emphasize  
의미: 강조하다  
품사: 동사  
예문: The manager emphasized the importance of clear communication in the workplace.  
예문해석: 관리자는 직장에서 명확한 커뮤니케이션의 중요성을 강조했다.  
발음기호: /ˈɛmfəˌsaɪz/  
---

---
단어: profound  


### 연애코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [5]:
print(dating_coach('저 이번 주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어요. 뭐가 좋을까요?'))

1. 상황분석:
   - 100일 기념일은 두 사람의 관계에서 중요한 마일스톤입니다. 특별한 날인 만큼, 여자친구에게 감동적이고 의미 있는 선물이 필요합니다. 여자친구의 취향이나 필요사항을 고려하여 개인적이고 특별한 선물을 준비하면 좋겠습니다.

2. 행동계획:
   - 여자친구의 관심사와 취향을 파악하기: 그녀가 좋아하는 것, 원했던 것, 또는 필요하다고 이야기한 것들이 있는지 생각해 보세요.
   - 이를 바탕으로 한 선물 아이디어 목록 작성: 로맨틱한 요소가 포함된 선물, 실용적인 선물, 그리고 추억이 담길 수 있는 경우의 수를 고려합니다.
   - 선물의 포장을 신경 쓰고, 카드를 작성해 마음속 진심을 적어 두세요.

3. 실행:
   - 선물 아이디어:
      - 개인적인 선물: 맞춤형 팔찌나 necklaces, 그녀의 이름이나 특별한 날짜가 각인된 제품.
      - 경험 선물: 근처의 맛집 예약, 둘만의 특별한 데이트 코스, 혹은 짧은 여행 티켓 등.
      - 감동적인 선물: 두 사람의 사진으로 만들어진 포토북이나 콜라주, 함께 쓴 편지.
   - 위의 아이디어들을 바탕으로 결정 후, 100일 기념일 당일에는 꽃다발이나 작은 선물로 시작하는 것도 좋은 방법입니다. 
   - 선물과 함께 진심 가득한 편지를 전하며, 예전의 순간들과 미래의 약속을 되새길 수 있는 시간을 만들어 보세요. 

이렇게 진행하신다면 그녀에게 잊지 못할 100일이 될 것입니다!


## 냉털 마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지 조언해준다.



In [14]:
def neng_tal(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
당신은 냉장고 속 재료를 기반으로 창의적이고 실용적인 요리법을 추천해주는 AI 요리사입니다.
사용자가 제공한 재료로 만들 수 있는 음식 중, 시간과 난이도, 맛을 고려하고, 단백질 등 영양분의 불균형이 없도록 최선의 선택을 안내해 주세요.
가능하다면 조리 팁, 보완할 재료도 함께 제안해 주세요. 마지막으로 해당 요리의 칼로리와 영양성분을 계산해보고 알려주세요.

<< Output Format >>
1. 상황분석: 재료 특성과 가능한 조합 설명

2. 행동계획: 만들 수 있는 요리명 + 이유 (2~3개 제안)

3. 실행: 레시피 (간단한 단계 중심)

4. 영양성분 : 칼로리 & 영양성분(단백질 등)
    """

    user_message = f"""
    냉장고에 남은 음식재료에 대한 사용자 입력:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [17]:
from IPython.display import Markdown


display(Markdown(neng_tal('냉장고에 계란이랑 밥이랑 햄, 당근, 대파 정도 있고, 닭가슴살이 있네. ')))

1. 상황분석: 
주어진 재료는 계란, 밥, 햄, 당근, 대파, 그리고 닭가슴살입니다. 이러한 재료를 통해 다양한 요리를 만들 수 있으며, 기본적인 단백질(계란, 햄, 닭가슴살), 탄수화물(밥), 비타민과 섬유소(당근, 대파)가 함께 포함되어 있습니다. 이를 기반으로 건강하고 맛있는 요리를 만들 수 있습니다.

2. 행동계획: 
   - 계란 볶음밥: 간단하게 만들 수 있으며, 단백질과 탄수화물을 균형 있게 섭취할 수 있습니다.
   - 닭가슴살 스크램블: 닭가슴살과 계란을 활용해 단백질을 강조한 요리를 만들 수 있습니다.
   - 햄과 당근을 넣은 오믈렛: 맛과 영양을 함께 잡을 수 있는 조리법입니다.
    
3. 실행:
   - **계란 볶음밥 레시피**:
     1. 대파와 당근을 잘게 썬다.
     2. 팬에 기름을 두르고 대파와 당근을 볶는다.
     3. 잘게 썬 햄을 추가하고 같이 볶는다.
     4. 남은 밥을 넣고 볶으면서 간을 맞춘다. 
     5. 계란을 풀어 팬에 넣고 잘 섞어가며 볶는다.
     6. 필요 시 소금과 후추로 간을 맞춰 완성.
     
   - **닭가슴살 스크램블 레시피**:
     1. 닭가슴살을 작게 썰어 소금과 후추로 마리네이드한다.
     2. 팬에 기름을 두르고 닭가슴살을 중불에서 익힌다.
     3. 닭이 거의 익으면 계란을 풀어 넣고 혼합하여 스크램블한다.
     4. 대파를 추가하고 간을 맞춰 완성.
  
   - **햄과 당근 오믈렛 레시피**:
     1. 계란 2개를 풀어 소금과 후추로 간을 한다.
     2. 팬에 기름을 두르고 잘게 썬 햄과 당근을 볶는다.
     3. 볶은 재료에 계란을 부어 중불에서 익힌다.
     4. 약 1-2분 후 오믈렛을 반으로 접어 완성.

4. 영양성분: 
- 예를 들어, 계란 볶음밥 한 접시 기준으로
  - 칼로리: 약 600 kcal
  - 단백질: 25g (계란, 닭가슴살, 햄에서)
  - 탄수화물: 75g (밥에서)
  - 지방: 20g (기름 및 햄에서)
  - 섬유소: 3g (당근, 대파에서)
  
이와 같은 조리법으로 다양한 영양분을 고루 섭취할 수 있으며, 간단한 조리법으로 만들 수 있습니다. 조리 시 원하는 채소나 추가 재료가 있다면 함께 활용해도 좋습니다!

In [21]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{
    "hard_skill": [
        {
            "question": "",
            "answer": "",
        }
    ],
    "soft_skill": [
        {
            "question": "",
            "answer": "",
        }
    ]
}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [23]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""

print(job_interview(job_posting))
output = job_interview(job_posting)

print(output)
print(type(output))

{
    "hard_skill": [
        {
            "question": "Python을 사용하여 AI 모델 배포를 위한 인프라를 구축할 때 어떤 라이브러리나 프레임워크를 사용할 수 있나요?",
            "answer": "Python에서는 Flask나 FastAPI와 같은 웹 프레임워크를 사용하여 API를 구축할 수 있고, TensorFlow Serving이나 BentoML과 같은 플랫폼을 사용하여 AI 모델을 서빙할 수 있습니다. MLflow는 모델 버전 관리와 실험 추적을 위한 유용한 도구입니다."
        },
        {
            "question": "컨테이너 오케스트레이션 툴인 Kubernetes를 사용한 경험에 대해 말씀해 주세요.",
            "answer": "Kubernetes를 사용하여 배포를 자동화하고, 로드 밸런싱 및 자원 관리를 자동으로 처리했습니다. 또한, Helm을 사용하여 패키지 관리를 효율적으로 수행하였고, 지속적인 배포를 위해 CI/CD 파이프라인을 구축하여 Jenkins와 통합했습니다."
        },
        {
            "question": "Jenkins 또는 ArgoCD를 사용한 CI/CD 파이프라인 구축 경험에 대해 설명해 주세요.",
            "answer": "Jenkins를 사용하여 Git 저장소에서 코드를 빌드하고 테스트하는 파이프라인을 구축했습니다. 배포 프로세스는 Jenkins의 외부 스크립트를 통해 Kubernetes 클러스터에 자동으로 배포되도록 설정했습니다. ArgoCD를 사용하여 GitOps 기반으로 Kubernetes 리소스를 관리하며, 선언적 방식으로 인프라를 유지할 수 있었습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "AI 모델을 배포하는 과정에서 팀원과의 커뮤니케이션을 어떻게 효과적으로 진

In [28]:
import json

data = json.loads(output)
print(type(data))
print(data)

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
{'hard_skill': [{'question': 'Python 프로그래밍과 관련하여 어떤 경험이 있나요?', 'answer': '저는 Python으로 데이터 분석과 머신러닝 모델 구현을 다수 경험해왔습니다. 특히, Pandas와 NumPy를 사용하여 데이터 전처리를 수행하고, Scikit-learn을 활용하여 다양한 알고리즘을 적용하여 모델을 학습시킨 경험이 있습니다. 또한, Flask를 이용하여 간단한 API를 구현해본 경험이 있어, 모델을 실제 환경에서 운용할 수 있는 기본적인 API 시스템 구축도 가능합니다.'}, {'question': 'Kubernetes와 Docker를 활용한 컨테이너 오케스트레이션에 대해 설명해 주세요.', 'answer': 'Kubernetes는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하는 오케스트레이션 플랫폼입니다. Docker는 애플리케이션을 컨테이너화 함으로써 이식성과 효율성을 개선하는 도구입니다. 저는 Docker로 이미지를 생성하고, Kubernetes 클러스터를 설정하여 여러 컨테이너를 조율한 경험이 있습니다. 이를 통해 ML 모델을 안정적으로 배포하고, 필요에 따라 스케일링할 수 있었습니다.'}, {'question': 'Jenkins 또는 GitLab CI/CD 파이프라인 구축 경험에 대해 설명해 주세요.', 'answer': '저는 Jenkins를 이용하여 CI/CD 파이프라인을 구성한 경험이 있습니다. 이를 통해 코드 변경 사항이 발생할 때마다 자동으로 테스트, 빌드, 배포가 이루어지도록 설정했습니다. GitLab에도 비슷한 파이프라인을 구성하여, GitLab CI/CD를 활용해 원활한 배포 프로세스를 구현했습니다. 이 과정에서 발생할 수 있는 오류를 조기에 발견하고, 지속적인 배포를 통해 효율성을 높였습니다.'}], 'soft_skill': [{'question': '팀원 간의 기술 지원을 효과적으로 수행하기 위한 방법은 무엇인가요?', 'answer': '팀원 간